In [1]:
############################################################################################################################
## Script: AGOL_2_SQL.py
## Author: Jeremy Mullins, Derek Morgan
## Date:
##
## Description:
##        This script is the test script for importing a AGOL feature layer into MS SQL as a table.
##
## Required prerequisites:
##        - ArcGIS API for Python
##            (https://developers.arcgis.com/python/)
##
##        - pyodbc Module
##            (https://github.com/mkleehammer/pyodbc)
##                to install after download, open Python command prompt and type the following:
##                    -- conda install pyodbc  --
##
##        - MS ODBC Driver for SQL Server (v17)
##            (https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-2017)
##
############################################################################################################################

In [2]:
# import all necessary modules
from arcgis.gis import GIS
import pyodbc as sql
import pandas as pd
import configparser
import zipfile
import arcpy
import sys
import os

In [24]:
#read config file
config = configparser.ConfigParser()
config.read('config_jdm.ini')

# assign config variables
workSPACE = config['SCRIPT']['workSPACE']
agolURL = config['AGOL']['URL']
agolUSER = config['AGOL']['USER']
agolPW = config['AGOL']['PW']
iD = config['SCRIPT']['itemID']
csvTITLE = config['SCRIPT']['csvTITLE']
csvLOC = config['SCRIPT']['csvOUTPUT']
zipLOC = config['SCRIPT']['zipLOC']
sqlDRVR = config['SQL']['odbcDRVR']
sqlSERV = config['SQL']['SERVER']
sqlDB = config['SQL']['DB']
sqlUSER = config['SQL']['USER']
sqlPW = config['SQL']['PW']
createTABLE = config['SQL']['createTEMP']
csvINSERT = config['SQL']['csvINSERT']
insNewRows = config['SQL']['updateTABLE']
delTEMP = config['SQL']['delTEMP']

In [4]:
#set workspace(s)
arcpy.env.workspace = workSPACE
os.chdir(workSPACE)

In [5]:
# sign into AGOL account
gis = GIS(agolURL, agolUSER, agolPW)

In [6]:
# get feature layer in question
featureLayer = gis.content.get(iD)

In [8]:
# export feature layer into CSV and save locally as ZIP file
output_file = featureLayer.export(title=csvTITLE,
                                   export_format="CSV")
output_file.download(csvLOC)

'C:/Data/PhD/Projects/SRC/src/Notebooks\\test.zip'

In [9]:
# unzip downloaded feature layer data
zip_ref = zipfile.ZipFile(zipLOC, 'r')
zip_ref.extractall(csvLOC)
zip_ref.close()

In [10]:
# remove excess files
os.remove(zipLOC)
output_file.delete()

True

In [42]:
# import CSV into SQL database as view
conn = sql.connect('Driver='+sqlDRVR+';'
                      'Server='+sqlSERV+';'
                      'Database='+sqlDB+';'
                      'trusted_connection=yes;'
                      'UID='+sqlUSER+';'
                      'PWD='+sqlPW+';')
cursor = conn.cursor()

#https://datatofish.com/how-to-connect-python-to-sql-server-using-pyodbc/

In [32]:
# read CSV into pandas and fill any NaN with '000'
df = pd.read_csv('Buildings_0.csv')
df.fillna('000',inplace=True)

In [35]:
# creates temporary table
cursor.execute('CREATE TABLE [dbo].[src_temp]([OBJECTID] INT, '
               '[BuildingIdent] varchar(100), '
               '[Description] varchar(50), '
               '[Class] varchar(3), '
               '[TypeCode] INT, ' 
               '[GlobalID] varchar(50))')
cursor.commit()

In [36]:
# inserts rows from CSV
for index,row in df.iterrows():
    cursor.execute('INSERT INTO [dbo].[src_temp]([OBJECTID], '
                   '[BuildingIdent],[Description],[Class],[TypeCode],[GlobalID])  '
                   'values (?,?,?,?,?,?)',
                   row['OBJECTID'],row['Building Identifier'],row['Description'],row['Classrooms'],row['TypeCode'],row['GlobalID'])
cursor.commit()

# https://tomaztsql.wordpress.com/2018/07/15/using-python-pandas-dataframe-to-read-and-insert-data-to-microsoft-sql-server/

In [44]:
# executes insert into production table, from temp table, loading only new records
cursor.execute('INSERT INTO [dbo].[src_prod](OBJECTID, '
               'BuildingIdent,Description,Class,TypeCode,GlobalID) '  
               'SELECT * FROM [dbo].[src_temp] UNION ' 
               'SELECT * FROM [dbo].[src_prod] EXCEPT ' 
               'SELECT * FROM [dbo].[src_temp] INTERSECT SELECT * FROM [dbo].[src_prod]')
cursor.commit()

# deletes temporary table
#cursor.execute(delTEMP)
cursor.commit()

In [40]:
# commits all previous executes, closes and deletes cursor and closes connection
cursor.close()
del cursor
conn.close()